## 소개
이 예에서는 사용자 정의 데이터 세트에서 의미론적 세그먼테이션을 수행하기 위해 SegFormer 모델 변형을 미세 조정하는 방법을 보여줍니다. 시맨틱 세그먼테이션은 이미지의 모든 픽셀에 카테고리를 할당하는 작업입니다. 세그포머는 세그포머에서 제안되었습니다: 트랜스포머를 사용한 시맨틱 세그먼테이션을 위한 간단하고 효율적인 설계에서 제안되었습니다. SegFormer는 계층적 트랜스포머 아키텍처("믹스 트랜스포머"라고 함)를 인코더와 경량 디코더로 사용하여 세그먼테이션을 수행합니다. 그 결과, 기존 모델보다 효율적이면서 시맨틱 세그먼테이션에 대한 최첨단 성능을 제공합니다. 자세한 내용은 원본 논문을 참조하세요.

![segformer-arch](https://i.imgur.com/BsrVwYe.png)

우리는 다음을 활용합니다.
[허깅 페이스 트랜스포머](https://github.com/huggingface/transformers)
를 활용하여 사전 학습된 세그포머 체크포인트를 로드하고 사용자 정의 데이터 세트에서 미세 조정합니다.

**참고: 이 예제에서는 다음 소스의 코드를 재사용합니다:

* [텐서플로 팀의 세분화 공식 튜토리얼](https://www.tensorflow.org/tutorials/images/segmentation)
* [세분화에 대한 허깅 페이스 태스크 가이드](https://huggingface.co/docs/transformers/main/en/tasks/semantic_segmentation)

이 예제를 실행하려면 '트랜스포머' 라이브러리를 설치해야 합니다:

In [ ]:
# Transformers 라이브러리를 설치합니다.
# 이 라이브러리는 Hugging Face에서 자연어 처리 모델과 관련된 다양한 모델과 도구를 제공합니다.
# - 'transformers'는 라이브러리 이름입니다.
# - '-q' 옵션은 설치 과정에서 출력되는 로그 메시지를 간소화합니다.
!pip install transformers -q

## 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the data

We use the [Oxford-IIIT Pets](https://www.robots.ox.ac.uk/~vgg/data/pets/) dataset for
this example. We leverage `tensorflow_datasets` to load the dataset.

In [ ]:
import tensorflow_datasets as tfds

# 'tensorflow_datasets' 라이브러리를 tfds로 별칭 지정하여 import합니다.

# "oxford_iiit_pet:3.*.*"는 로드하려는 데이터셋의 이름을 나타냅니다.
# Oxford-IIIT Pet 데이터셋은 애완 동물 이미지 데이터셋입니다.

# 'with_info=True'는 데이터셋에 대한 추가 정보를 함께 로드하겠다는 옵션입니다.
# 이 옵션을 사용하면 데이터셋의 정보, 예를 들어 클래스 수, 이미지 크기 등을 가져올 수 있습니다.

dataset, info = tfds.load("oxford_iiit_pet:3.*.*", with_info=True)

# 'tfds.load()' 함수를 사용하여 데이터셋을 로드하고, 결과를 'dataset'과 'info' 변수에 할당합니다.
# 'dataset'에는 실제 데이터가 들어가고, 'info'에는 데이터셋에 대한 정보가 들어갑니다.

## 데이터 세트 준비하기

훈련 및 평가를 위한 데이터 세트를 준비합니다:

* 사전 훈련 중에 사용된 평균과 표준 편차로 이미지를 정규화합니다.
SegFormer.
* 픽셀 값이 0부터 시작하도록 세그먼테이션 마스크에서 1을 뺍니다.
* 이미지의 크기를 조정합니다.
* 이미지가 `"channels_first"` 형식이 되도록 이미지의 위치를 바꿉니다. 이렇게 하면
포옹 얼굴 트랜스포머의 세그포머 모델과 호환되도록 하기 위함입니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend

# 필요한 라이브러리를 import합니다.
# 'tensorflow'는 TensorFlow 라이브러리입니다.
# 'backend' 모듈은 Keras의 백엔드 관련 기능을 제공합니다.

# 이미지 사이즈와 정규화를 위한 평균과 표준편차를 정의합니다.
image_size = 512
mean = tf.constant([0.485, 0.456, 0.406])
std = tf.constant([0.229, 0.224, 0.225])

# 이미지를 정규화하는 함수를 정의합니다.
def normalize(input_image, input_mask):
    # 이미지를 부동 소수점 타입으로 변환합니다.
    input_image = tf.image.convert_image_dtype(input_image, tf.float32)

    # 평균으로 빼고 표준편차로 나누어 정규화합니다.
    input_image = (input_image - mean) / tf.maximum(std, backend.epsilon())

    # 레이블(마스크)에서 1을 빼서 클래스를 0부터 시작하도록 만듭니다.
    input_mask -= 1

    return input_image, input_mask

# 데이터를 로드하고 전처리하는 함수를 정의합니다.
def load_image(datapoint):
    # 이미지와 마스크의 크기를 조정합니다.
    input_image = tf.image.resize(datapoint["image"], (image_size, image_size))
    input_mask = tf.image.resize(
        datapoint["segmentation_mask"],
        (image_size, image_size),
        method="bilinear",
    )

    # 이미지를 정규화합니다.
    input_image, input_mask = normalize(input_image, input_mask)

    # 이미지의 차원 순서를 변경합니다. (채널 순서 변경)
    input_image = tf.transpose(input_image, (2, 0, 1))

    # 출력값을 딕셔너리 형태로 반환합니다.
    return {"pixel_values": input_image, "labels": tf.squeeze(input_mask)}

이제 위의 유틸리티를 사용하여 `tf.data.Dataset` 객체를 준비합니다.
'prefetch()`를 사용하여 준비합니다. 훈련에 사용하는 GPU의 메모리 크기와 일치하도록 `batch_size`를 변경합니다.
에 맞게 변경합니다.

In [ ]:
auto = tf.data.AUTOTUNE
batch_size = 4

# 훈련 데이터셋 전처리 및 배치화
train_ds = (
    dataset["train"]  # 'train' 데이터셋을 선택합니다.
    .cache()  # 데이터를 캐싱하여 반복적인 데이터 로드를 최소화합니다.
    .shuffle(batch_size * 10)  # 셔플을 통해 데이터를 섞습니다.
    .map(load_image, num_parallel_calls=auto)  # 'load_image' 함수를 병렬로 적용하여 전처리합니다.
    .batch(batch_size)  # 배치 크기를 설정합니다.
    .prefetch(auto)  # 데이터를 미리 로드하여 학습 속도를 높입니다.
)

# 테스트 데이터셋 전처리 및 배치화
test_ds = (
    dataset["test"]  # 'test' 데이터셋을 선택합니다.
    .map(load_image, num_parallel_calls=auto)  # 'load_image' 함수를 병렬로 적용하여 전처리합니다.
    .batch(batch_size)  # 배치 크기를 설정합니다.
    .prefetch(auto)  # 데이터를 미리 로드하여 학습 속도를 높입니다.
)

입력 이미지의 모양과 세분화 맵을 확인할 수 있습니다:

In [ ]:
# 훈련 데이터셋의 요소(spec)에 대한 정보를 출력합니다.
print(train_ds.element_spec)

## Visualize dataset

In [ ]:
import matplotlib.pyplot as plt

# 이미지를 시각적으로 표시하는 함수를 정의합니다.
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ["Input Image", "True Mask", "Predicted Mask"]

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis("off")
    plt.show()


# 훈련 데이터셋에서 2개의 샘플을 가져와서 시각적으로 표시합니다.
for samples in train_ds.take(2):
    sample_image, sample_mask = samples["pixel_values"][0], samples["labels"][0]

    # 이미지의 차원 순서를 변경합니다. (채널 순서 변경)
    sample_image = tf.transpose(sample_image, (1, 2, 0))

    # 마스크의 차원을 변경하여 시각적으로 표시할 수 있도록 준비합니다.
    sample_mask = tf.expand_dims(sample_mask, -1)

    # display 함수를 사용하여 이미지와 마스크를 시각적으로 표시합니다.
    display([sample_image, sample_mask])

## 사전 학습된 세그포머 체크포인트 로드하기

이제 허깅 페이스 트랜스포머에서 사전 훈련된 세그포머 모델 변형을 로드합니다. 세그포머 모델은
세그포머 모델은 **MiT-B0**에서 **MiT-B5**로 불리는 다양한 변형이 있습니다. 여러분은
이 체크포인트를 찾아보세요.
[여기](https://huggingface.co/models?pipeline_tag=image-segmentation&sort=downloads&search=segformer).
가장 작은 변형인 Mix-B0을 로드하는데, 이는 추론 효율성과 예측 성능 간의
좋은 트레이드 오프를 제공합니다.

In [ ]:
from transformers import TFSegformerForSemanticSegmentation

# 사용할 모델 체크포인트를 지정합니다. 이 경우 "nvidia/mit-b0"를 사용합니다.
model_checkpoint = "nvidia/mit-b5"

# 클래스에 대한 ID와 레이블 매핑을 정의합니다.
id2label = {0: "outer", 1: "inner", 2: "border"}
label2id = {label: id for id, label in id2label.items()}

# 클래스의 수를 계산합니다.
num_labels = len(id2label)

# TFSegformerForSemanticSegmentation 모델을 불러오고 초기화합니다.
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,  # 클래스 수를 지정합니다.
    id2label=id2label,      # ID에서 레이블로의 매핑을 설정합니다.
    label2id=label2id,      # 레이블에서 ID로의 매핑을 설정합니다.
    ignore_mismatched_sizes=True,  # 입력 크기가 모델에 맞지 않아도 무시합니다.
)

경고는 일부 가중치를 버리고 새로 초기화 중임을 알려줍니다.
초기화 중이라는 경고입니다. 당황하지 마세요! 이는 지극히 정상적인 현상입니다. 사용자 정의 데이터 세트를 사용하고 있기 때문에
사전 학습 데이터 세트와 다른 시맨틱 클래스 레이블 집합을 가진 사용자 정의 데이터 세트를 사용하고 있기 때문입니다,
[`TFSegformerForSemanticSegmentation`](https://huggingface.co/docs/transformers/model_doc/segformer#transformers.TFSegformerForSemanticSegmentation)
는 새로운 디코더 헤드를 초기화합니다.

이제 최적화기를 초기화하고 모델을 컴파일할 수 있습니다.

## Compile the model

In [ ]:
lr = 0.00006  # 학습률을 설정합니다.
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)  # Adam 옵티마이저를 사용합니다.
model.compile(optimizer=optimizer)  # 모델을 컴파일합니다.

모델을 컴파일할 때 손실 함수를 사용하지 않는다는 점에 유의하세요. 그 이유는
모델의 포워드 패스
[구현](https://github.com/huggingface/transformers/blob/820c46a707ddd033975bc3b0549eea200e64c7da/src/transformers/models/segformer/modeling_tf_segformer.py#L873)
모델의 포워드 패스는 입력 이미지와 함께 레이블을 제공할 때 손실 계산 부분을 처리하기 때문입니다. 손실 계산 후
손실 계산 후, 모델은 구조화된 '데이터 클래스' 객체를 반환합니다.
구조화된 데이터 클래스를 반환합니다.

컴파일된 모델을 가지고 `fit()`을 호출하여 미세 조정을 시작할 수 있습니다.
프로세스를 시작할 수 있습니다!

## 훈련 진행 상황을 모니터링하기 위한 예측 콜백

모델을 미세 조정할 때 몇 가지 샘플 예측을 시각화하는 데 도움이 됩니다,
모델의 진행 상황을 모니터링하는 데 도움이 됩니다. 이 콜백은 다음에서 영감을 얻었습니다.
[이 튜토리얼](https://www.tensorflow.org/tutorials/images/segmentation).

In [ ]:
from IPython.display import clear_output

# 모델의 예측 결과에서 가장 확률이 높은 클래스를 선택하여 마스크를 생성하는 함수를 정의합니다.
def create_mask(pred_mask):
    pred_mask = tf.math.argmax(pred_mask, axis=1)
    pred_mask = tf.expand_dims(pred_mask, -1)
    return pred_mask[0]

# 모델의 예측 결과를 시각적으로 확인하는 함수를 정의합니다.
def show_predictions(dataset=None, num=1):
    if dataset:
        for sample in dataset.take(num):
            images, masks = sample["pixel_values"], sample["labels"]
            masks = tf.expand_dims(masks, -1)
            pred_masks = model.predict(images).logits
            images = tf.transpose(images, (0, 2, 3, 1))
            display([images[0], masks[0], create_mask(pred_masks)])
    else:
        display(
            [
                sample_image,
                sample_mask,
                create_mask(model.predict(tf.expand_dims(sample_image, 0))),
            ]
        )

# 학습 중에 콜백을 통해 예측 결과를 시각적으로 확인할 수 있도록 하는 클래스를 정의합니다.
class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset, **kwargs):
        super().__init__(**kwargs)
        self.dataset = dataset

    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions(self.dataset)
        print("\nSample Prediction after epoch {}\n".format(epoch + 1))

## Train model

In [ ]:
# 에폭 수를 설정합니다. 만약 결과가 기대한 품질이 아니라면 에폭 수를 늘려보세요.
epochs = 5

# 모델을 학습합니다.
history = model.fit(
    train_ds,  # 훈련 데이터셋을 사용합니다.
    validation_data=test_ds,  # 검증 데이터셋을 사용합니다.
    callbacks=[DisplayCallback(test_ds)],  # 학습 중에 예측 결과를 시각적으로 확인하는 콜백을 사용합니다.
    epochs=epochs,  # 학습 에폭 수를 설정합니다.
)

## 추론

테스트 세트의 몇 가지 샘플에 대해 추론을 수행합니다.

In [ ]:
# 테스트 데이터셋에서 5개의 샘플에 대한 예측 결과를 시각적으로 확인합니다.
show_predictions(test_ds, 5)

## 결론

이 예제에서는 의미론적 세분화를 위한 사용자 정의
데이터 세트에서 세그포머 모델 변형을 미세 조정하는 방법을 배웠습니다. 간결성을 위해 예제는
는 짧게 작성했습니다. 하지만 몇 가지 추가적으로 시도해 볼 수 있는 몇 가지 사항이 있습니다:

* 데이터 보강을 통합하여 잠재적으로 결과를 개선합니다.
* 더 큰 세그포머 모델 체크포인트를 사용하여 결과가 어떻게 영향을 받는지 확인합니다.
* 미세 조정된 모델을 허깅 페이스에 푸시하여 커뮤니티와 쉽게 공유할 수 있습니다.
model.push_to_hub("your-username/your-awesome-model")`를 실행하면 됩니다.
그런 다음 다음을 수행하여 모델을 로드할 수 있습니다.
TFSegformerForSemanticSegmentation.from_pretrained("your-username/your-awesome-model"`).
[여기](https://github.com/huggingface/notebooks/blob/main/examples/semantic_segmentation-tf.ipynb)
는 참조를 찾고 있다면 엔드 투 엔드 예제입니다.
* 모델이 미세 조정되는 동안 모델 체크포인트를 허브에 푸시하고 싶다면
대신 `PushToHubCallback` Keras 콜백을 사용할 수 있습니다.
[여기](https://gist.github.com/sayakpaul/f474ffb01f0cdcc8ba239357965c3bca)가 그 예입니다.
[여기](https://huggingface.co/sayakpaul/mit-b0-finetuned-pets)는 모델을 사용하여 생성된 모델
리포지토리의 예입니다.